In [1]:
import sys
sys.path.append('../bhnerf')
import os
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import utils, emission_utils, visualization, network_utils, observation_utils
import ehtim as eh
import ipyvolume as ipv
from ipywidgets import interact
import ipywidgets as widgets

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-a3mc8xfq because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



## Plots for illustration figure

In [21]:
initial_frame = emission_utils.generate_hotspots_2d(256,256,np.pi, 3.5, std=0.4)

orbit_period = 3.5**(-3./2.) 
velocity_field = lambda r: (1.0 / orbit_period) * r**(-3/2)
gaussian = emission_utils.generate_orbit_2d(initial_frame, 128, velocity_field)


In [38]:
%matplotlib widget
(gaussian.isel(t=0) + gaussian.isel(t=20) + gaussian.isel(t=40)).where(gaussian.x**2 + gaussian.y**2 > 1).plot(cmap='afmhot', vmin=0.1, rasterized=True)
plt.savefig('figures/gaussian.pdf', transparent=False, bbox_inches=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [63]:
sensor = xr.load_dataset('../sensors/a0.00_th1.57_ngeo200_npix200.nc')
sensor = sensor.where(sensor.r < 10)

In [117]:
pixel = sensor.sel(pix=164)
%matplotlib widget
plt.plot(pixel.x, pixel.y, color='tab:blue')

pixel = sensor.sel(pix=32)
plt.plot(pixel.x, pixel.y, color='tab:green')

pixel = sensor.sel(pix=35)
plt.plot(pixel.x, pixel.y, color='tab:red')

meas = emission_utils.integrate_rays(gaussian.isel(t=0), sensor)
plt.plot(15-3*meas, np.linspace(-8, 8, sensor.pix.size), color='tab:blue')

meas = emission_utils.integrate_rays(gaussian.isel(t=20), sensor)
plt.plot(20-3*meas, np.linspace(-8, 8, sensor.pix.size), color='tab:green')

meas = emission_utils.integrate_rays(gaussian.isel(t=40), sensor)
plt.plot(25-3*meas, np.linspace(-8, 8, sensor.pix.size), color='tab:red')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [104]:
plt.savefig('figures/rays.pdf', transparent=False, bbox_inches=0)

## Plot UV coverage

In [2]:
obs_params = {
    'array': eh.array.load_txt('../eht_arrays/ngEHT.txt'),
    'nt': 128,      # number of time samples 
    'tstart': 4.0,  # start of observations (UTC hr)
    'tstop': 15.5,  # end of observation (UTC hr)
    'tint': 60.0    # integration time
}
obs_ng = observation_utils.empty_eht_obs(**obs_params)

obs_params = {
    'array': eh.array.load_txt('../eht_arrays/EHT2017.txt'),
    'nt': 128,      # number of time samples 
    'tstart': 4.0,  # start of observations (UTC hr)
    'tstop': 15.5,  # end of observation (UTC hr)
    'tint': 60.0    # integration time
}
obs_2017 = observation_utils.empty_eht_obs(**obs_params)

In [11]:
%matplotlib widget
fig, ax = plt.subplots(1,2, figsize=(8,4), sharey=True)
observation_utils.plot_uv_coverage(obs_2017, ax=ax[0], fontsize=12)
observation_utils.plot_uv_coverage(obs_ng, ax=ax[1], fontsize=12)
ax[1].set_ylabel('')
ax[0].set_title('EHT 2017 Sampling', fontsize=14)
ax[1].set_title('ngEHT Sampling', fontsize=14)
plt.savefig('figures/eht_sampling.png', transparent=False, bbox_inches=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …